<a href="https://colab.research.google.com/github/codingcat101/TextToImageAndVideoGeneration/blob/main/texttoimagevideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Persist Ventures intern task*


#Text to Image Generation


In [1]:
!pip install diffusers
!pip install transformers scipy ftfy accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00


In [3]:
import torch
import os
from diffusers import StableDiffusionPipeline
from PIL import Image, ImageDraw, ImageFont
import matplotlib.font_manager as fm


pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
pipe = pipe.to("cuda")

In [5]:


def generate_motivational_image(prompt, text, idx, output_dir="output"):
    image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

    # Add motivational text
    draw = ImageDraw.Draw(image)
    font_path = fm.findfont(fm.FontProperties(family="DejaVu Sans"))
    font = ImageFont.truetype(font_path, size=int(image.size[1] / 15))

    # Calculate text position
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
    position = ((image.size[0] - text_width) // 2, image.size[1] - text_height - 20)

    # Add a semi-transparent rectangle
    margin = 10
    rectangle_position = [
        position[0] - margin, position[1] - margin,
        position[0] + text_width + margin, position[1] + text_height + margin
    ]
    draw.rectangle(rectangle_position, fill=(0, 0, 0, 128))
    draw.text(position, text, (255, 255, 255), font=font)

    # Save the image
    os.makedirs(output_dir, exist_ok=True)
    file_name = os.path.join(output_dir, f"motivational_image_{idx}.png")
    image.save(file_name)
    print(f"Image {idx + 1} saved as {file_name}")
    return image

# Generate multiple images
prompt = "a beautiful sunrise over mountains"
motivational_text = "Rise and Shine!"
num_images = 5
output_dir = "motivational_images"

images = []
for i in range(num_images):
    print(f"Generating image {i + 1}...")
    try:
        image = generate_motivational_image(prompt, motivational_text, i, output_dir)
        images.append(image)
    except Exception as e:
        print(f"Error generating image {i + 1}: {e}")

# Display images (if supported)
for idx, img in enumerate(images):
    img.show()


Generating image 1...


  0%|          | 0/50 [00:00<?, ?it/s]

Image 1 saved as motivational_images/motivational_image_0.png
Generating image 2...


  0%|          | 0/50 [00:00<?, ?it/s]

Image 2 saved as motivational_images/motivational_image_1.png
Generating image 3...


...

Image 3 saved as motivational_images/motivational_image_2.png
Generating image 4...


  0%|          | 0/50 [00:00<?, ?it/s]

Image 4 saved as motivational_images/motivational_image_3.png
Generating image 5...


  0%|          | 0/50 [00:00<?, ?it/s]

Image 5 saved as motivational_images/motivational_image_4.png






## Text-to-Video Generation


In [6]:
pip install diffusers transformers accelerate torch


In [ ]:
pip install diffusers

In [7]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import HTML
from base64 import b64encode
import datetime

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()


!mkdir /content/videos

model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

text_encoder/config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

model.fp16.safetensors:   0%|          | 0.00/681M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import numpy as np
import cv2
import os
import datetime

# Prepare directories
os.makedirs('/content/videos', exist_ok=True)

# Load the pipeline
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

# Define a function to generate and save a video
def generate_video(prompt, video_index):
    print(f"Generating video {video_index + 1} for prompt: {prompt}")

    # Generate video frames
    output = pipe(prompt, num_inference_steps=25, num_frames=24)
    video_frames = output.frames

    # Process frames
    def process_frames(frames):
        if len(frames.shape) == 5:  # Case: (1, num_frames, height, width, channels)
            frames = frames[0]
        elif len(frames.shape) != 4:
            raise ValueError(f"Unexpected frame shape: {frames.shape}")

        # Normalize and convert
        frames = ((frames - frames.min()) / (frames.max() - frames.min()) * 255).astype(np.uint8)
        return frames

    processed_frames = process_frames(video_frames)

    # Save video using OpenCV
    output_video_path = f'/content/videos/temp_video_{video_index}.mp4'
    height, width, layers = processed_frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video_path, fourcc, 8, (width, height))

    for frame in processed_frames:
        video.write(frame)
    video.release()

    # Post-process with FFmpeg
    timestamped_path = f'/content/videos/{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}_video_{video_index}.mp4'
    os.system(f"ffmpeg -y -i {output_video_path} -c:v libx264 -c:a aac -strict -2 {timestamped_path} >/dev/null 2>&1")

    print(f"Video {video_index + 1} saved to: {timestamped_path}")
    return timestamped_path

# Accept a single text prompt from the user
prompt = input("Enter your motivational text prompt: ")

# Generate 5 videos from the single prompt
video_paths = []
for i in range(5):
    video_path = generate_video(prompt, i)
    video_paths.append(video_path)

# Display all videos
from IPython.display import Video, display
for video_path in video_paths:
    display(Video(video_path, embed=True))


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Enter your motivational text prompt: "Show a person planting a seed, watering it, and watching it grow into a small plant over time."
Generating video 1 for prompt: "Show a person planting a seed, watering it, and watching it grow into a small plant over time."


  0%|          | 0/25 [00:00<?, ?it/s]

Video 1 saved to: /content/videos/2024-12-18_08-57-32_video_0.mp4
Generating video 2 for prompt: "Show a person planting a seed, watering it, and watching it grow into a small plant over time."


  0%|          | 0/25 [00:00<?, ?it/s]

...

Video 3 saved to: /content/videos/2024-12-18_08-58-38_video_2.mp4
Generating video 4 for prompt: "Show a person planting a seed, watering it, and watching it grow into a small plant over time."


  0%|          | 0/25 [00:00<?, ?it/s]

Video 4 saved to: /content/videos/2024-12-18_08-59-11_video_3.mp4
Generating video 5 for prompt: "Show a person planting a seed, watering it, and watching it grow into a small plant over time."


  0%|          | 0/25 [00:00<?, ?it/s]

Video 5 saved to: /content/videos/2024-12-18_08-59-44_video_4.mp4


In [9]:
!cp {new_video_path} /content/videos/tmp.mp4
mp4 = open('/content/videos/tmp.mp4','rb').read()

decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src="{decoded_vid}" type="video/mp4"></video>')